# Find lifts wear and tear
How much more will one lift wear out compared to the other.

In my builing we have 2 lifts with one button that calls the closest one.
Noticed that when both of them are on the same floor, the left one will arrive.
I guess there is some chip that prioritese the first lift in the list (the left one: [0, 1]).

As a result left one gets more wear and tear and e.g. already has more issues with doors.
So I wonder — how much more left one will be worn out over time?

Main issue to check: piciking the first lift in the list, if both are on the same floor.

### TL;DR
Left one does ~2x more work:
```
Lift 4606717504: at 5, wear: 494535
Lift 4613068176: at 0, wear: 226344
```


### Plan
- call lift for random room
- move lift from its current floor to one where it's called
- move lift to destination
- from non 0 floor move to 0
- from 0 back to room's floor


In [81]:
import random

In [82]:
floors = 10
rooms_per_floor = 100

In [83]:
class Lift:
    wear = 0
    current_floor = 0
    
    def run_floors(self, n):
        self.wear += n

    def move(self, _from, _to):
        self.run_floors(abs(_from - _to))
        self.current_floor = _to

    def __repr__(self):
        return f'Lift {id(self)}: at {self.current_floor}, wear: {self.wear}'


class Room:
    floor: int
    number: int
    current_floor = 0
    trips = 0

    def __init__(self, floor, number):
        self.floor = floor
        self.number = number

    def __repr__(self):
        return f'Room: {self.floor}-{self.number}, {self.current_floor}, {self.trips}'

In [84]:
def pick_lift(floor):
    
    def distance(_l):
        return abs(floor-_l.current_floor)
        
    if len({l.current_floor for l in lifts}) == 1:
        closest = lifts[0]
    else:
        lifts_with_distance = [(l, distance(l)) for l in lifts]
        closest = sorted(lifts_with_distance, key=lambda x: x[1])[0][0]
    
    return closest

In [85]:
def call_lift(room):
    if room.current_floor == 0:
        _from = 0
        _to = room.floor
    else:
        _from = room.floor
        _to = 0
    lift = pick_lift(_from)
    lift.move(lift.current_floor, _from)
    lift.move(_from, _to)
    room.current_floor = _to
    room.trips += 1

## Run

In [86]:
lifts = (Lift(), Lift())

rooms = []
for i in range(1, floors):
    for j in range(rooms_per_floor):
        rooms.append(Room(floor=i, number=j))

In [87]:
for i in range(100000):
    room = random.choice(rooms)
    call_lift(room)

for lift in lifts:
    print(lift)

Lift 4606717504: at 5, wear: 494535
Lift 4613068176: at 0, wear: 226344


## Test

In [88]:
floors = 5
lifts = [Lift(), Lift()]
rooms = []
for i in range(1, floors):
    for j in range(rooms_per_floor):
        rooms.append(Room(floor=i, number=j))


lifts[0].current_floor = 4
# 4 vs 0
assert pick_lift(0) == lifts[1]  # 4-0 vs 0-0
assert pick_lift(1) == lifts[1]  # 4-1 vs 0-1
assert pick_lift(2) == lifts[0]  # 4-2 vs 0-2
assert pick_lift(3) == lifts[0]  # 4-3 vs 0-3
assert pick_lift(4) == lifts[0]  # 4-4 vs 1-0

# 4 vs 4
lifts[1].current_floor = 4
assert pick_lift(0) == lifts[0]


room = random.choice(rooms)
call_lift(room)
# move lift from 4 to 0 and to the room's floor
assert lifts[0].wear == room.floor + 4


# total trips
for i in range(10):
    room = random.choice(rooms)
    call_lift(room)

assert sum(r.trips for r in rooms) == 11